## Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [191]:
# -*- coding: utf8 -*-
import codecs
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import pickle
import networkx as nx
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False
from __future__ import print_function

In [192]:
import yaml

In [193]:
from IPython.html.widgets import FloatProgress
from IPython.display import display

In [194]:
import datetime
def dateheure():
    return datetime.datetime.utcnow().strftime('%y%m%d%H%M')

In [195]:
saut="\n"

### Préparation des matrices de traits

In [196]:
features.add_config('bdlexique.ini')
fs=features.FeatureSystem('phonemes')

In [197]:
sampleNumber="03-N"
sampleType="BRASS"
sampleType="SILVER"
genDigraphe=False
genGraphe=False
#samplePrefix="MGC-150916-extend-%s-paradigmes"%sampleNumber
#samplePrefix="MGC-160104-%s-VerbesActions-SILVER"%sampleNumber
#samplePrefix="MGC-160104-%s-ext3-paradigmes"%sampleNumber
#samplePrefix="MGC-160104-%s-ext3-derivations-BRASS"%sampleNumber
#samplePrefix="MGC-160104-derivation2-%s-SILVER"%sampleNumber
#samplePrefix="MGC-160104-derivation2-derives2"
#samplePrefix="MGC-160104-%s-basesDerives"%sampleNumber
#samplePrefix="MGC-160215-DerivesExtraits"
samplePrefix="MGC-160104-%s-ext3-160215-derivations-%s"%(sampleNumber,sampleType)
sampleFile=samplePrefix+".csv"
#goldFile="MGC-160104-01-N-VerbesActions-GOLD.csv"
#goldFile="MGC-160104-01-N-Gold.csv"
#analysisPrefix="MGC-150916-extend-%s"%sampleNumber
#analysisPrefix="MGC-160104-%s-VerbesActions-SILVER"%sampleNumber
#analysisPrefix="MGC-160104-%s-ext3-paradigmes"%sampleNumber
#analysisPrefix="MGC-160104-%s-ext3-derivations-BRASS"%sampleNumber
#analysisPrefix="MGC-160104-derivation2-%s-SILVER"%sampleNumber
#analysisPrefix="MGC-160215-DerivesExtraits"
analysisPrefix=samplePrefix
logfile_name=analysisPrefix+samplePrefix+".log"
logfile = codecs.open("2015-Data/"+logfile_name,mode='w',encoding="utf8")

In [198]:
paradigmes=pd.read_csv("2015-Data/"+sampleFile,sep=";",encoding="utf8")
del paradigmes[u"Unnamed: 0"]
paradigmes=paradigmes.dropna(axis=1,how='all')

In [199]:
phonologicalMap=analysisPrefix[-2:]
if debug: print(phonologicalMap)
neutralisationsNORD=(u"6û",u"9ê")
neutralisationsSUD=(u"e2o",u"E9O")
if phonologicalMap=="-N":
    neutralisations=neutralisationsNORD
elif phonologicalMap=="-S":
    neutralisations=neutralisationsSUD
else:
    neutralisations=(u"",u"")
    phonologicalMap=("-X")
bdlexiqueIn = unicode(u"èò"+neutralisations[0])
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO"+neutralisations[1])
neutralise = dict(zip(bdlexiqueNum, neutreOut))

In [200]:
cliqueFile="2015-Data/"+analysisPrefix+'-Network.pkl'
#paradigmes


In [201]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    else:
        result=chaine
    return result

In [202]:
goldCases=paradigmes.columns.tolist()
goldCases.remove("lexeme")
#goldCases

- sampleCases pour la liste des cases effectivement représentées dans le corpus de départ 

In [203]:
sampleCases=paradigmes.columns.values.tolist()
sampleCases.remove(u"lexeme")
#sampleCases

### Définition des objets pour la gestion des règles

In [204]:
class paireClasses:
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classes1=classesPaire(case1,case2)
        self.classes2=classesPaire(case2,case1)

    def ajouterPatron(self,n,patron,motif):
        if n==1:
            self.classes1.ajouterPatron(patron,motif)
        elif n==2:
            self.classes2.ajouterPatron(patron,motif)
        else:
            print ("le numéro de forme n'est pas dans [1,2]",n,file=logfile)

    def ajouterPaire(self,forme1,forme2):
        self.classes1.ajouterPaire(forme1,forme2)
        self.classes2.ajouterPaire(forme2,forme1)
        
    def calculerClasses(self):
        return(self.classes1,self.classes2)

    
class classesPaire:
    '''
    Gestion des patrons, des classes et des transformations
    
    ajouterPatron : ajoute un patron et son motif associé (MGL)
    ajouterPaire : ajoute une paire de formes, calcule la classe de la forme1 et la règle sélectionnée
    sortirForme : cacule les formes de sortie correspondant à la forme1 avec leurs coefficients respectifs
    '''
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classe={}
        self.nbClasse={}
        self.patrons={}
        self.entree={}
        self.sortie={}
    
    def ajouterPatron(self,patron,motif):
        self.patrons[patron]=motif
        (entree,sortie)=patron.split("-")
        self.entree[patron]=entree.replace(u".",u"(.)")
        self.sortie[patron]=remplacementSortie(sortie)
    
    def ajouterPaire(self,forme1,forme2):
        '''
        on calcule la classe de la paire idClasseForme et la règle sélectionnée
        on incrémente le compteur de la classe et celui de la règle sélectionnée à l'intérieur de la classe
        '''
        classeForme=[]
        regleForme=""
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme1):
                classeForme.append(patron)
                '''
                le +"$" permet de forcer l'alignement à droite pour les transformations suffixales
                '''
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleForme=patron
        idClasseForme=", ".join(classeForme)
        if not idClasseForme in self.classe:
            self.classe[idClasseForme]={}
            self.nbClasse[idClasseForme]=0
        if not regleForme in self.classe[idClasseForme]:
            self.classe[idClasseForme][regleForme]=0
        self.nbClasse[idClasseForme]+=1
        self.classe[idClasseForme][regleForme]+=1

    def sortirForme(self,forme):
        classeForme=[]
        sortieForme={}
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme):
                classeForme.append(patron)
        if classeForme:
            idClasseForme=", ".join(classeForme)
            if idClasseForme in self.nbClasse:
                nTotal=self.nbClasse[idClasseForme]
                for patron in self.classe[idClasseForme]:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(self.classe[idClasseForme][patron])/nTotal
            else:
                if debug:
                    print (forme, file=logfile)
                    print ("pas de classe",idClasseForme, file=logfile)
                    print ("%.2f par forme de sortie" % (float(1)/len(classeForme)), file=logfile)
                nTotal=len(classeForme)
                for patron in classeForme:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(1)/nTotal
        else:
            print (forme, file=logfile) 
            print ("pas de patron", file=logfile)
        return sortieForme
        

### Comparer les cases analysées avec l'ensemble de toutes les cases

In [205]:
with open("2015-Data/"+analysisPrefix+'-Regles.pkl', 'rb') as input:
    resultatsLecture = pickle.load(input)

In [206]:
analyseCases=list(set([case for (case,autre) in resultatsLecture.keys() if autre in ["FP"]]))
print (sorted(analyseCases))
#    analyseCases=sampleCases

[u'FP', u'FS', u'ai1P', u'ai1S', u'ai2P', u'ai2S', u'ai3P', u'ai3S', u'fi1P', u'fi1S', u'fi2P', u'fi2S', u'fi3P', u'fi3S', u'ii1P', u'ii1S', u'ii2P', u'ii2S', u'ii3P', u'ii3S', u'inf', u'is1P', u'is1S', u'is2P', u'is2S', u'is3P', u'is3S', u'pI1P', u'pI2P', u'pI2S', u'pP', u'pc1P', u'pc1S', u'pc2P', u'pc2S', u'pc3P', u'pc3S', u'pi1P', u'pi1S', u'pi2P', u'pi2S', u'pi3P', u'pi3S', u'ppFP', u'ppFS', u'ppMP', u'ppMS', u'ps1P', u'ps1S', u'ps2P', u'ps2S', u'ps3P', u'ps3S']


#### Calculer le score de la clique

In [207]:
def cliqueScore(clique,graph):
    score=0
    for (depart,arrivee) in it.combinations_with_replacement(clique,2):
        score+=graph[depart][arrivee]["weight"]
    return score

## Génération des formes

In [208]:
basesDerives=pd.read_csv("2015-Data/MGC-160104-01-N-basesDerives.csv",sep=";",encoding="utf8")

In [209]:
with open("2015-Data/"+analysisPrefix+'-Network.pkl', 'rb') as input:
    infoCliques=pickle.load(input)
cliques=infoCliques["cliques"]
cliquesScores=infoCliques["cliquesScores"]
cliquesListes=infoCliques["cliquesListes"]

In [210]:
#cliques

In [211]:
listeTest=list(set(case[0].split("-")[0] for case in cliques))
len(listeTest)

195

In [212]:
analyseCases=list(set(case.split("-")[-1] for clique in cliques for case in clique ))
#sorted(analyseCases)

In [213]:
longueurCliques={}
for clique in cliques:
    longueur=len(clique)
    if not longueur in longueurCliques:
        longueurCliques[longueur]=1
    else:
        longueurCliques[longueur]+=1
longueurCliques

{1: 1,
 2: 145,
 3: 607,
 4: 862,
 5: 1078,
 6: 1758,
 7: 1782,
 8: 2204,
 9: 2377,
 10: 2695,
 11: 2759,
 12: 2651,
 13: 2331,
 14: 2778,
 15: 2694,
 16: 2799,
 17: 2504,
 18: 1921,
 19: 1862,
 20: 1790,
 21: 1328,
 22: 1243,
 23: 1159,
 24: 1298,
 25: 964,
 26: 896,
 27: 920,
 28: 542,
 29: 508,
 30: 451,
 31: 369,
 32: 284,
 33: 342,
 34: 242,
 35: 218,
 36: 184,
 37: 228,
 38: 157,
 39: 101,
 40: 89,
 41: 83,
 42: 72,
 43: 60,
 44: 62,
 45: 16,
 46: 18,
 47: 6}

### Faire la liste des cases lexicalisées de l'échantillon

In [214]:
nbFormesLexicales={}
casesLexicales={element:paradigmes[paradigmes["lexeme"]==element].columns[paradigmes[paradigmes["lexeme"]==element].notnull().iloc[0]].tolist() for element in listeTest}
for element in casesLexicales:
    casesLexicales[element].remove("lexeme")
    nbFormesLexicales[element]=len(casesLexicales[element])
#nbFormesLexicales

In [215]:
def cliques2Tableau(cliques):
    paradigmesListe=[]
    for n,clique in enumerate(cliques):
        paradigmeClique={}
        for element in clique:
            elements=element.split("-")
            lexeme=elements[0]
            nbInitial=len(casesLexicales[lexeme])
            forme=elements[-2]
            taminfo=elements[-1]
            paradigmeClique[taminfo]=forme
        paradigmeClique["lexeme"]=lexeme
        paradigmeClique["score"]=cliquesScores[n]
        paradigmeClique["ajouts"]=len(clique)-nbInitial
        paradigmesListe.append(paradigmeClique)
    return pd.DataFrame(paradigmesListe,columns=["lexeme"]+analyseCases+[u"score",u"ajouts"])


In [216]:
cliquesPANDAS=cliques2Tableau(cliques)

In [217]:
filtreDERIVE=~(cliquesPANDAS["FS"].isnull() & cliquesPANDAS["FP"].isnull())
cliquesDERIVES=cliquesPANDAS[filtreDERIVE]

In [218]:
def fideliteLexique(numero,clique):
    formesLexicales=[]
    paradigmeClique={}
    formesPresentes=[]
    formesAbsentes=[]
    #####################
    #Prépararer la représentation lexicale
    #####################
    #
    # transformer les cases lexicales en points
    # au format clique pour comparaison
    #
    #####################

    #####################
    #Identifier le lexème
    #####################
    point=clique[0].split("-")
    lPoint=len(point)
    if lPoint==3:
        lexeme=point[0]
    else:
        lexeme="-".join(point[0:len(point)-2])
    paradigmeClique["lexeme"]=lexeme
    #####################
    #Nombre de cases lexicales
    #####################
    casesLexeme=casesLexicales[lexeme]
    nbCasesLexicales=len(casesLexeme)
    nbFormesLexicales=nbCasesLexicales
    #####################
    #Transformation des cases lexicales
    #####################
    if casesLexeme:
        for element in casesLexeme:
            champForme=paradigmes[paradigmes["lexeme"]==lexeme][element].iloc[0]
            if ","  in champForme:
                formes=champForme.split(",")
                nbFormesLexicales+=len(formes)-1
                for forme in formes:
                    pointCase=u"%s-%s-%s"% (lexeme,forme,element)
                    formesLexicales.append(pointCase)
                    if pointCase in clique:
                        formesPresentes.append(pointCase)
                    else:
                        formesAbsentes.append(pointCase)
            else:
                forme=champForme
                pointCase=u"%s-%s-%s"% (lexeme,forme,element)
                formesLexicales.append(pointCase)
                if pointCase in clique:
                    formesPresentes.append(pointCase)
                else:
                    formesAbsentes.append(pointCase)

    #######################################
    #
    # Compter les formes fidèles
    # Préparer le résultat
    #
    #######################################
    for element in clique:
        elements=element.split("-")
        forme=elements[-2]
        taminfo=elements[-1]
        paradigmeClique[taminfo]=forme
    paradigmeClique["score"]=cliquesScores[numero]
    paradigmeClique["ajouts"]=len(clique)-len(formesPresentes)
    paradigmeClique["lexOk"]=len(formesPresentes)
    paradigmeClique["lexMiss"]=len(formesAbsentes)
    return (paradigmeClique,formesPresentes,formesAbsentes,formesLexicales)

## Comparer la sortie des cliques avec le paradigme de départ

In [219]:
%%time
seuilClique=1
#paradigmesCLIQUES=pd.DataFrame(columns=paradigmes.columns.values.tolist())
paradigmesListe=[]
#for n,clique in enumerate(sorted(cliques,key=lambda x: len(x),reverse=True)):
progressBar = FloatProgress(min=0, max=len(cliques))
display(progressBar)
for n,clique in enumerate(cliques):
    progressBar.value=n
#    if seuilClique==0:
#        seuilClique=len(clique)-15
    if len(clique)>seuilClique:
        (paradigmeClique,formesPresentes,formesAbsentes,formesLexicales)=fideliteLexique(n,clique)
        paradigmesListe.append(paradigmeClique)
#paradigmesCLIQUES=pd.DataFrame(paradigmesListe,columns=GOLD.columns.values.tolist()+[u"score",u"ajouts"])
paradigmesCLIQUES=pd.DataFrame(paradigmesListe,columns=["lexeme"]+analyseCases+[u"score",u"ajouts",u"lexOk",u"lexMiss"])
#print (seuilClique)

CPU times: user 16min 41s, sys: 8.19 s, total: 16min 49s
Wall time: 16min 45s


In [220]:
paradigmesCLIQUES

,lexeme,FP,ii1P,ii1S,FS,is2S,is1S,pP,ai3S,ppMP,...,fi3S,fi3P,ai3P,fi1P,fi1S,pI1P,score,ajouts,lexOk,lexMiss
0,absoudre,NaN,NaN,absOlvE,NaN,NaN,NaN,absOlvâ,absOlva,absOlve,...,absOlv9ra,absOlv9rô,NaN,absOlv9rô,absOlv9rE,NaN,84.962938,18,2,8
1,absoudre,NaN,NaN,absOlvE,NaN,NaN,NaN,absOlvâ,absOlva,absOlve,...,absOlv9ra,absOlv9rô,NaN,absOlv9rô,absOlv9rE,NaN,84.619379,15,3,7
2,absoudre,NaN,NaN,absOlvE,NaN,NaN,NaN,absOlvâ,absOlva,absOlve,...,absOlv9ra,absOlv9rô,absOlvEr,absOlv9rô,absOlv9rE,absOlvô,102.165063,20,2,8
3,absoudre,NaN,NaN,absOlvE,NaN,NaN,NaN,absOlvâ,absOlva,absOlve,...,absOlv9ra,absOlv9rô,absOlvEr,absOlv9rô,absOlv9rE,absOlvô,101.760545,17,3,7
4,absoudre,NaN,NaN,absOlvE,NaN,NaN,NaN,absOlvâ,absOlva,absOlve,...,absOlv9ra,absOlv9rô,absOlvEr,NaN,absOlv9rE,absOlvô,132.949373,23,2,8
5,absoudre,NaN,NaN,absOlvE,NaN,NaN,NaN,absOlvâ,absOlva,absOlve,...,NaN,NaN,absOlvEr,NaN,NaN,absOlvô,113.573532,21,2,8
6,absoudre,NaN,absOlvjô,absOlvE,NaN,NaN,NaN,absOlvâ,absOlva,absOlve,...,NaN,NaN,absOlvEr,NaN,NaN,absOlvô,136.128034,23,2,8
7,absoudre,NaN,NaN,absOlvE,NaN,NaN,NaN,absOlvâ,absOlva,absOlve,...,absOlv9ra,absOlv9rô,absOlvEr,NaN,absOlv9rE,absOlvô,132.311327,20,3,7
8,absoudre,NaN,NaN,absOlvE,NaN,NaN,NaN,absOlvâ,absOlva,absOlve,...,NaN,NaN,absOlvEr,NaN,NaN,absOlvô,113.097005,18,3,7
9,absoudre,NaN,absOlvjô,absOlvE,NaN,NaN,NaN,absOlvâ,absOlva,absOlve,...,NaN,NaN,absOlvEr,NaN,NaN,absOlvô,135.424463,20,3,7


In [221]:
filtreCLIQUES=~(paradigmesCLIQUES["FS"].isnull() & paradigmesCLIQUES["FP"].isnull())
paradigmesFILTRES=paradigmesCLIQUES[filtreCLIQUES]

In [222]:
def maxColonne(table,listeGroupe,colonne):
    return table[table.groupby(listeGroupe)[colonne].transform(max)==table[colonne]]

In [223]:
paradigmesMaxLex=maxColonne(paradigmesFILTRES,["lexeme"],"lexOk")
paradigmesMaxScore=maxColonne(paradigmesMaxLex,["lexeme"],"score")
paradigmeMAX=paradigmesMaxScore
paradigmeMAX[["lexeme","FS","FP"]]

,lexeme,FS,FP
35,absoudre,absut,absut
98,accalmir,akalmire,akalmire
314,accroître,akrwase,akrwase
1226,agonir,agOnise,agOnise
1834,aller,ale,ale
2102,amener,amEne,amEne
2172,arriver,arive,arive
2809,asseoir,asiz,asiz
3000,astreindre,astrêt,astrêt
3127,atteindre,atêt,atêt


In [224]:
paradigmeSILVER=pd.merge(paradigmes,paradigmeMAX,on=["lexeme"])

In [225]:
filtreNouveauxDerivesSILVER=(paradigmeSILVER["FS_x"].isnull() & paradigmeSILVER["FP_x"].isnull())
filtreFsLexSILVER=(paradigmeSILVER["FS_x"].notnull() &
    (paradigmeSILVER["FS_x"]!=paradigmeSILVER["FS_y"]) 
    )
filtreFpLexSILVER=(paradigmeSILVER["FP_x"].notnull() &
    (paradigmeSILVER["FP_x"]!=paradigmeSILVER["FP_y"]) 
    )
paradigmeSILVER[(~filtreNouveauxDerivesSILVER)&(filtreFsLexSILVER)][["lexeme","FS_x","FP_x","FS_y","FP_y"]]
paradigmeSILVER[(~filtreNouveauxDerivesSILVER)&(filtreFpLexSILVER)][["lexeme","FS_x","FP_x","FS_y","FP_y"]]
paradigmeSILVER[filtreNouveauxDerivesSILVER][["lexeme","FS_y","FP_y"]]


,lexeme,FS_y,FP_y
1,accalmir,akalmire,akalmire
2,accroître,akrwase,akrwase
3,agonir,agOnise,agOnise
5,amener,amEne,amEne
11,avancer,avâse,avâse
12,battre,bate,bate
13,bramer,brame,brame
14,branler,brâle,brâle
17,buter,byte,byte
18,charpir,Sarpit,Sarpit


In [226]:
paradigmeSILVER[filtreNouveauxDerivesSILVER][["lexeme","FS_y","FP_y"]].to_csv(path_or_buf="2015-Data/"+sampleFile.replace(".csv","-NouveauxDerives.csv"),encoding="utf8",sep=";")

In [227]:
paradigmeSILVER[["lexeme","FS_x","FP_x","FS_y","FP_y"]].to_csv(path_or_buf="2015-Data/"+sampleFile.replace(".csv","-TousDerives.csv"),encoding="utf8",sep=";")

In [228]:
with open("2015-Data/"+sampleFile.replace(".csv","-TousDerives.pkl"), 'wb') as output:
   pickle.dump(paradigmeSILVER, output, pickle.HIGHEST_PROTOCOL)